<a href="https://colab.research.google.com/github/hijadelena/call-center-analytics-python-tableau/blob/main/EDA_fiber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**🧩Dashboard de Análisis de Llamadas Repetidas al Servicio de Atención al Cliente**

1. **Limpieza y carga de datos, los pasos que hacemos**

Cargar los datasets desde Google Sheets

Estandarizar nombres de columnas

Eliminar duplicados y valores nulos

Convertir columnas de fecha

Verificar tipos de datos

Añadir columnas calculadas útiles para Tableau



In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

def clean_google_fiber_data():
    """
    Limpieza mejorada de datos de Google Fiber con manejo robusto de nulos
    y agregación de fechas duplicadas
    """

    # 1. Cargar datos desde Google Sheets
    print("📊 Cargando datos de Google Sheets...")
    url_market1 = 'https://docs.google.com/spreadsheets/d/1T7irtn0ay9MfuhG_6y2jEbeT-zLuEQTbv1qNxkpBaTE/export?format=csv'
    url_market2 = 'https://docs.google.com/spreadsheets/d/1RE_MAKPt0JfWijbYCR32XFra9OGlmS2tLMxNs-X2NSU/export?format=csv'

    try:
        df1 = pd.read_csv(url_market1)
        df2 = pd.read_csv(url_market2)
        print(f"✅ Market 1: {len(df1)} registros")
        print(f"✅ Market 2: {len(df2)} registros")
    except Exception as e:
        print(f"❌ Error cargando datos: {e}")
        return None

    # 2. Estandarizar nombres de columnas
    print("\n🔧 Estandarizando columnas...")
    df1.columns = df1.columns.str.strip().str.lower().str.replace(' ', '_')
    df2.columns = df2.columns.str.strip().str.lower().str.replace(' ', '_')

    # 3. Verificar estructura común
    if set(df1.columns) != set(df2.columns):
        print("⚠️ Diferencias en columnas:")
        print(f"Solo en Market 1: {set(df1.columns) - set(df2.columns)}")
        print(f"Solo en Market 2: {set(df2.columns) - set(df1.columns)}")
        # Tomar columnas comunes
        common_cols = list(set(df1.columns) & set(df2.columns))
        df1 = df1[common_cols]
        df2 = df2[common_cols]

    # 4. Unir dataframes
    print("🔗 Uniendo datasets...")
    df = pd.concat([df1, df2], ignore_index=True)
    print(f"Dataset combinado: {len(df)} registros")

    # 5. Análisis inicial de nulos
    print("\n📋 Análisis inicial de valores nulos:")
    null_counts = df.isnull().sum()
    null_percentages = (df.isnull().sum() / len(df)) * 100
    null_summary = pd.DataFrame({
        'Nulos': null_counts,
        'Porcentaje': null_percentages
    }).sort_values('Porcentaje', ascending=False)
    print(null_summary[null_summary['Nulos'] > 0])

    # 6. Limpiar y convertir fecha (initial attempt)
    print("\n📅 Procesando fechas...")
    if 'date_created' in df.columns:
        # Probar múltiples formatos de fecha
        date_formats = ['%Y-%m-%d', '%d/%m/%Y', '%m/%d/%Y', '%Y-%m-%d %H:%M:%S']
        df['date_created_clean'] = None

        for fmt in date_formats:
            mask = df['date_created_clean'].isnull()
            try:
                df.loc[mask, 'date_created_clean'] = pd.to_datetime(
                    df.loc[mask, 'date_created'],
                    format=fmt,
                    errors='coerce'
                )
                converted = mask.sum() - df['date_created_clean'].isnull().sum()
                if converted > 0:
                    print(f"✅ Convertidas {converted} fechas con formato {fmt}")
            except:
                continue

        # Eliminar registros con fechas inválidas
        invalid_dates = df['date_created_clean'].isnull().sum()
        if invalid_dates > 0:
            print(f"⚠️ Eliminando {invalid_dates} registros con fechas inválidas")
            df = df.dropna(subset=['date_created_clean'])

        df['date_created'] = df['date_created_clean']
        df = df.drop('date_created_clean', axis=1)
        # Ensure date_created is datetime after initial cleaning
        df['date_created'] = pd.to_datetime(df['date_created'], errors='coerce')


    # 7. Identificar y limpiar columnas de contactos
    print("\n☎️ Procesando columnas de contactos...")
    contact_columns = [col for col in df.columns if 'contacts_' in col]
    print(f"Columnas de contactos encontradas: {contact_columns}")

    # Convertir a numérico y llenar nulos con 0
    for col in contact_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    # 8. Limpiar columnas categóricas
    print("\n🏷️ Limpiando columnas categóricas...")
    categorical_columns = []

    # Identificar columnas de tipo/mercado
    for col in df.columns:
        if any(keyword in col.lower() for keyword in ['type', 'market', 'new_']):
            categorical_columns.append(col)

    for col in categorical_columns:
        if col in df.columns:
            # Limpiar espacios y valores vacíos
            df[col] = df[col].astype(str).str.strip()
            df[col] = df[col].replace(['', 'nan', 'None', 'null'], np.nan)

            # Mostrar valores únicos para revisión
            unique_values = df[col].value_counts(dropna=False)
            print(f"\n{col} - Valores únicos:")
            print(unique_values.head(10))

    # 9. Manejar fechas duplicadas - AGREGACIÓN
    print("\n📊 Manejando fechas duplicadas...")

    # Definir columnas de agrupación (excluyendo contactos)
    group_columns = ['date_created']
    if 'new_market' in df.columns:
        group_columns.append('new_market')
    if 'new_type' in df.columns:
        group_columns.append('new_type')

    # Ensure group_columns are in df.columns before grouping
    group_columns = [col for col in group_columns if col in df.columns]

    # Convert date_created to datetime before checking for duplicates
    df['date_created'] = pd.to_datetime(df['date_created'], errors='coerce')


    # Identify duplicates before aggregation
    duplicates_before = df.duplicated(subset=group_columns).sum()
    print(f"Registros duplicados por fecha/mercado/tipo: {duplicates_before}")

    if duplicates_before > 0:
        print("🔄 Agregando datos por fecha/mercado/tipo...")

        # Agregación: sumar columnas de contactos
        agg_dict = {}
        for col in contact_columns:
            agg_dict[col] = 'sum'

        # Maintain first occurrence for other columns, excluding date_created
        for col in df.columns:
            if col not in contact_columns + group_columns and col != 'date_created':
                agg_dict[col] = 'first'

        df_aggregated = df.groupby(group_columns, as_index=False).agg(agg_dict)
        print(f"✅ Datos agregados: {len(df)} → {len(df_aggregated)} registros")
        df = df_aggregated
        # Ensure date_created is datetime after aggregation
        df['date_created'] = pd.to_datetime(df['date_created'], errors='coerce')


    # 10. Renombrar columnas para claridad
    print("\n🏷️ Renombrando columnas...")
    rename_mapping = {
        'new_type': 'problem_type',
        'new_market': 'market'
    }
    df = df.rename(columns=rename_mapping)

    # 11. Calcular métricas clave
    print("\n📈 Calculando métricas...")

    # Identificar columnas de llamadas repetidas
    repeat_columns = [col for col in contact_columns if '_n_' in col and col != 'contacts_n']

    # Calcular totales
    df['total_repeats'] = df[repeat_columns].sum(axis=1)
    df['total_calls'] = df['contacts_n'] + df['total_repeats']
    df['resolved_first_call'] = (df['total_repeats'] == 0).astype(int)
    df['repeat_call_flag'] = (df['total_repeats'] > 0).astype(int)

    # Calcular tasa de resolución en primera llamada
    if len(df) > 0:
        fcr_rate = (df['resolved_first_call'].sum() / len(df)) * 100
        repeat_rate = (df['repeat_call_flag'].sum() / len(df)) * 100
        print(f"📊 FCR Rate: {fcr_rate:.2f}%")
        print(f"📊 Repeat Call Rate: {repeat_rate:.2f}%")

    # 12. Crear variables temporales para análisis
    print("\n📅 Creando variables temporales...")
    # Ensure date_created is datetime before using .dt accessor
    df['date_created'] = pd.to_datetime(df['date_created'], errors='coerce')
    df['year'] = df['date_created'].dt.year
    df['month'] = df['date_created'].dt.month
    df['week'] = df['date_created'].dt.isocalendar().week
    df['day_of_week'] = df['date_created'].dt.day_name()
    df['date_only'] = df['date_created'].dt.date

    # Períodos para agrupación
    df['year_month'] = df['date_created'].dt.to_period('M').astype(str)
    df['year_week'] = df['date_created'].dt.to_period('W').astype(str)

    # 13. Validación final
    print("\n✅ Validación final...")
    print(f"Registros finales: {len(df)}")
    print(f"Rango de fechas: {df['date_created'].min()} - {df['date_created'].max()}")
    print(f"Nulos restantes por columna:")

    final_nulls = df.isnull().sum()
    if final_nulls.sum() > 0:
        print(final_nulls[final_nulls > 0])
    else:
        print("Sin valores nulos ✅")

    # 14. Estadísticas descriptivas
    print("\n📊 Estadísticas descriptivas:")
    numeric_cols = ['contacts_n', 'total_repeats', 'total_calls']
    print(df[numeric_cols].describe())

    # 15. Exportar datos limpios
    output_file = 'google_fiber_clean.csv'
    df.to_csv(output_file, index=False)
    print(f"\n💾 Datos exportados a: {output_file}")

    # 16. Generar reporte de calidad
    generate_data_quality_report(df, output_file.replace('.csv', '_quality_report.txt'))

    return df

def generate_data_quality_report(df, filename):
    """Generar reporte de calidad de datos"""
    with open(filename, 'w', encoding='utf-8') as f:
        f.write("GOOGLE FIBER - REPORTE DE CALIDAD DE DATOS\n")
        f.write("=" * 50 + "\n\n")
        f.write(f"Fecha de generación: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

        f.write("1. RESUMEN GENERAL\n")
        f.write("-" * 20 + "\n")
        f.write(f"Total de registros: {len(df):,}\n")
        f.write(f"Total de columnas: {len(df.columns)}\n")
        f.write(f"Rango de fechas: {df['date_created'].min()} - {df['date_created'].max()}\n")
        f.write(f"Período total: {(df['date_created'].max() - df['date_created'].min()).days} días\n\n")

        f.write("2. DISTRIBUCIÓN POR MERCADO\n")
        f.write("-" * 30 + "\n")
        if 'market' in df.columns:
            market_dist = df['market'].value_counts()
            for market, count in market_dist.items():
                f.write(f"{market}: {count:,} registros ({count/len(df)*100:.1f}%)\n")
        f.write("\n")

        f.write("3. DISTRIBUCIÓN POR TIPO DE PROBLEMA\n")
        f.write("-" * 40 + "\n")
        if 'problem_type' in df.columns:
            type_dist = df['problem_type'].value_counts()
            for ptype, count in type_dist.items():
                f.write(f"{ptype}: {count:,} registros ({count/len(df)*100:.1f}%)\n")
        f.write("\n")

        f.write("4. MÉTRICAS DE RENDIMIENTO\n")
        f.write("-" * 30 + "\n")
        fcr_rate = (df['resolved_first_call'].sum() / len(df)) * 100
        repeat_rate = (df['repeat_call_flag'].sum() / len(df)) * 100
        avg_calls = df['total_calls'].mean()

        f.write(f"Tasa de resolución en primera llamada (FCR): {fcr_rate:.2f}%\n")
        f.write(f"Tasa de llamadas repetidas: {repeat_rate:.2f}%\n")
        f.write(f"Promedio de llamadas por caso: {avg_calls:.2f}\n\n")

        f.write("5. COLUMNAS DISPONIBLES PARA TABLEAU\n")
        f.write("-" * 40 + "\n")
        for col in sorted(df.columns):
            dtype = str(df[col].dtype)
            nulls = df[col].isnull().sum()
            f.write(f"{col} ({dtype}) - Nulos: {nulls}\n")

if __name__ == "__main__":
    print("🚀 Iniciando limpieza de datos Google Fiber...")
    clean_data = clean_google_fiber_data()
    print("\n🎉 Proceso completado!")

🚀 Iniciando limpieza de datos Google Fiber...
📊 Cargando datos de Google Sheets...
✅ Market 1: 450 registros
✅ Market 2: 450 registros

🔧 Estandarizando columnas...
🔗 Uniendo datasets...
Dataset combinado: 900 registros

📋 Análisis inicial de valores nulos:
              Nulos  Porcentaje
contacts_n_6    445   49.444444
contacts_n_5    439   48.777778
contacts_n_7    435   48.333333
contacts_n_4    432   48.000000
contacts_n_3    404   44.888889
contacts_n_2    374   41.555556
contacts_n_1    328   36.444444
contacts_n      132   14.666667

📅 Procesando fechas...
✅ Convertidas 900 fechas con formato %Y-%m-%d

☎️ Procesando columnas de contactos...
Columnas de contactos encontradas: ['contacts_n', 'contacts_n_1', 'contacts_n_2', 'contacts_n_3', 'contacts_n_4', 'contacts_n_5', 'contacts_n_6', 'contacts_n_7']

🏷️ Limpiando columnas categóricas...

new_type - Valores únicos:
new_type
type_5    180
type_1    180
type_2    180
type_4    180
type_3    180
Name: count, dtype: int64

new_market

-----CORRECCION DE METRICAS----
se encontro que el fcr y otras metricas tenian, Error en el código Python:
pythondf['resolved_first_call'] = (df['total_repeats'] == 0).astype(int)
Esta lógica considera que si total_repeats == 0, entonces se resolvió en primera llamada. Pero esto es incorrecto porque:

Un registro puede tener contacts_n = 0 (sin primera llamada) y total_repeats = 0
Esto marcaría erróneamente como "resuelto en primera llamada"\

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

def complete_corrected_analysis(df):
    """
    Análisis completo con TODAS las métricas corregidas + análisis de patrones
    """

    # ==========================================
    # 1. CORRECCIÓN COMPLETA DE MÉTRICAS
    # ==========================================

    print("🔧 CORRIGIENDO TODAS LAS MÉTRICAS...")
    print("=" * 50)

    # Identificar columnas de contactos
    contact_columns = [col for col in df.columns if 'contacts_n' in col]
    repeat_columns = [col for col in contact_columns if 'contacts_n_' in col]

    print(f"📞 Columna primera llamada: contacts_n")
    print(f"🔁 Columnas repetidas: {repeat_columns}")

    # Calcular totales CORRECTOS
    df['first_calls'] = df['contacts_n'].fillna(0)
    df['repeat_n1'] = df['contacts_n_1'].fillna(0) if 'contacts_n_1' in df.columns else 0
    df['repeat_n2'] = df['contacts_n_2'].fillna(0) if 'contacts_n_2' in df.columns else 0
    df['repeat_n3'] = df['contacts_n_3'].fillna(0) if 'contacts_n_3' in df.columns else 0
    df['repeat_n4'] = df['contacts_n_4'].fillna(0) if 'contacts_n_4' in df.columns else 0
    df['repeat_n5'] = df['contacts_n_5'].fillna(0) if 'contacts_n_5' in df.columns else 0
    df['repeat_n6'] = df['contacts_n_6'].fillna(0) if 'contacts_n_6' in df.columns else 0
    df['repeat_n7'] = df['contacts_n_7'].fillna(0) if 'contacts_n_7' in df.columns else 0

    # Total repetidas y total llamadas
    df['total_repeats'] = df[repeat_columns].sum(axis=1)
    df['total_calls'] = df['first_calls'] + df['total_repeats']

    # ==========================================
    # 2. MÉTRICAS CORREGIDAS (COMO TABLEAU)
    # ==========================================

    # Totales agregados
    total_first_calls = df['first_calls'].sum()
    total_repeat_calls = df['total_repeats'].sum()
    total_all_calls = df['total_calls'].sum()

    # FCR y Repeat Rate (como Tableau)
    fcr_rate = (1 - (total_repeat_calls / total_all_calls)) * 100 if total_all_calls > 0 else 0
    repeat_rate = (total_repeat_calls / total_all_calls) * 100 if total_all_calls > 0 else 0

    # Casos únicos vs llamadas totales
    total_cases = len(df)
    cases_no_repeat = (df['total_repeats'] == 0).sum()
    cases_with_repeat = (df['total_repeats'] > 0).sum()

    print("\n📊 MÉTRICAS PRINCIPALES (CORREGIDAS):")
    print("-" * 40)
    print(f"Total de llamadas: {total_all_calls:,}")
    print(f"├─ Primera llamada: {total_first_calls:,}")
    print(f"└─ Llamadas repetidas: {total_repeat_calls:,}")
    print(f"FCR Rate: {fcr_rate:.1f}%")
    print(f"Repeat Rate: {repeat_rate:.1f}%")
    print(f"Casos totales: {total_cases:,}")
    print(f"├─ Sin repetición: {cases_no_repeat:,} ({cases_no_repeat/total_cases*100:.1f}%)")
    print(f"└─ Con repetición: {cases_with_repeat:,} ({cases_with_repeat/total_cases*100:.1f}%)")

    # ==========================================
    # 3. ANÁLISIS DE ESCALAMIENTO (N1 → N7)
    # ==========================================

    print("\n🔍 ANÁLISIS DE ESCALAMIENTO POR CONTACTO:")
    print("-" * 45)

    escalamiento_data = []
    for i, col in enumerate(repeat_columns, 1):
        if col in df.columns:
            total_contacto = df[col].sum()
            pct_del_total = (total_contacto / total_repeat_calls * 100) if total_repeat_calls > 0 else 0
            escalamiento_data.append({
                'Contacto': f'N{i}',
                'Llamadas': total_contacto,
                'Porcentaje': pct_del_total
            })
            print(f"Contacto N{i}: {total_contacto:,} llamadas ({pct_del_total:.1f}%)")

    escalamiento_df = pd.DataFrame(escalamiento_data)

    # Detectar picos de escalamiento
    if len(escalamiento_df) > 0:
        max_contacto = escalamiento_df.loc[escalamiento_df['Llamadas'].idxmax()]
        print(f"\n🎯 Pico máximo en: {max_contacto['Contacto']} con {max_contacto['Llamadas']:,} llamadas")

        # Identificar aumentos inusuales (como el N6 que mencionas)
        aumentos = []
        for i in range(1, len(escalamiento_df)):
            anterior = escalamiento_df.iloc[i-1]['Llamadas']
            actual = escalamiento_df.iloc[i]['Llamadas']
            if actual > anterior:
                aumento_pct = ((actual - anterior) / anterior * 100) if anterior > 0 else 0
                aumentos.append(f"N{i+1} (+{aumento_pct:.1f}% vs N{i})")

        if aumentos:
            print(f"⚠️  Aumentos detectados en: {', '.join(aumentos)}")

    # ==========================================
    # 4. ANÁLISIS POR TIPO DE PROBLEMA
    # ==========================================

    print("\n📋 ANÁLISIS POR TIPO DE PROBLEMA:")
    print("-" * 35)

    # Mapeo de tipos (según tu información)
    type_mapping = {
        'type_1': 'Account Manager',
        'type_2': 'Technical',
        'type_3': 'Scheduling',
        'type_4': 'Construction',
        'type_5': 'Internet/WiFi'
    }

    if 'problem_type' in df.columns:
        # Aplicar mapeo si es necesario
        df['problem_type_label'] = df['problem_type'].map(type_mapping).fillna(df['problem_type'])

        problem_analysis = df.groupby('problem_type_label').agg({
            'first_calls': 'sum',
            'total_repeats': 'sum',
            'total_calls': 'sum',
            'problem_type': 'count'  # casos
        }).rename(columns={'problem_type': 'casos'})

        problem_analysis['fcr_pct'] = (1 - (problem_analysis['total_repeats'] / problem_analysis['total_calls'])) * 100
        problem_analysis['repeat_rate'] = (problem_analysis['total_repeats'] / problem_analysis['total_calls']) * 100
        problem_analysis['avg_calls_per_case'] = problem_analysis['total_calls'] / problem_analysis['casos']

        # Ordenar por problemas más problemáticos
        problem_analysis = problem_analysis.sort_values('repeat_rate', ascending=False)

        print("\nRanking por Mayor Tasa de Repetición:")
        for idx, row in problem_analysis.iterrows():
            print(f"{idx}:")
            print(f"  • Llamadas repetidas: {row['total_repeats']:,} ({row['repeat_rate']:.1f}%)")
            print(f"  • FCR: {row['fcr_pct']:.1f}%")
            print(f"  • Promedio llamadas/caso: {row['avg_calls_per_case']:.1f}")

        # Análisis de escalamiento por tipo
        print(f"\n🔍 ESCALAMIENTO POR TIPO DE PROBLEMA:")
        escalamiento_por_tipo = []

        for problema in df['problem_type_label'].unique():
            if pd.notna(problema):
                subset = df[df['problem_type_label'] == problema]
                escalamiento_tipo = {}

                for i, col in enumerate(repeat_columns, 1):
                    if col in subset.columns:
                        escalamiento_tipo[f'N{i}'] = subset[col].sum()

                # Buscar picos en N6 específicamente
                if 'N6' in escalamiento_tipo and len(escalamiento_tipo) > 5:
                    n5_val = escalamiento_tipo.get('N5', 0)
                    n6_val = escalamiento_tipo.get('N6', 0)
                    n7_val = escalamiento_tipo.get('N7', 0)

                    if n6_val > n5_val and n6_val > n7_val and n6_val > 0:
                        pct_aumento = ((n6_val - n5_val) / n5_val * 100) if n5_val > 0 else 0
                        print(f"  ⚠️  {problema}: Pico en N6 ({n6_val:,} llamadas, +{pct_aumento:.1f}% vs N5)")

                escalamiento_por_tipo.append({
                    'Problema': problema,
                    **escalamiento_tipo,
                    'Total_Repeats': sum(escalamiento_tipo.values())
                })

        escalamiento_tipo_df = pd.DataFrame(escalamiento_por_tipo)
        if not escalamiento_tipo_df.empty:
            escalamiento_tipo_df = escalamiento_tipo_df.sort_values('Total_Repeats', ascending=False)

    # ==========================================
    # 5. ANÁLISIS POR MERCADO
    # ==========================================

    print(f"\n🏙️ ANÁLISIS POR MERCADO:")
    print("-" * 25)

    if 'market' in df.columns:
        market_analysis = df.groupby('market').agg({
            'first_calls': 'sum',
            'total_repeats': 'sum',
            'total_calls': 'sum',
            'market': 'count'  # casos
        }).rename(columns={'market': 'casos'})

        market_analysis['fcr_pct'] = (1 - (market_analysis['total_repeats'] / market_analysis['total_calls'])) * 100
        market_analysis['repeat_rate'] = (market_analysis['total_repeats'] / market_analysis['total_calls']) * 100

        print("Comparación entre mercados:")
        for mercado, row in market_analysis.iterrows():
            print(f"{mercado}:")
            print(f"  • FCR: {row['fcr_pct']:.1f}%")
            print(f"  • Tasa repetidas: {row['repeat_rate']:.1f}%")
            print(f"  • Total llamadas: {row['total_calls']:,}")

        # Identificar mercado con mejores/peores prácticas
        mejor_mercado = market_analysis.loc[market_analysis['fcr_pct'].idxmax()]
        peor_mercado = market_analysis.loc[market_analysis['fcr_pct'].idxmin()]

        print(f"\n🏆 Mejor FCR: {mejor_mercado.name} ({mejor_mercado['fcr_pct']:.1f}%)")
        print(f"⚠️  Peor FCR: {peor_mercado.name} ({peor_mercado['fcr_pct']:.1f}%)")

        # Análisis cruzado: Mercado vs Tipo de Problema
        print(f"\n🔍 ANÁLISIS CRUZADO MERCADO vs PROBLEMA:")
        if 'problem_type_label' in df.columns:
            cross_analysis = df.pivot_table(
                values='total_repeats',
                index='market',
                columns='problem_type_label',
                aggfunc='sum',
                fill_value=0
            )

            print("Llamadas repetidas por mercado y problema:")
            print(cross_analysis)

            # Identificar combinaciones problemáticas
            max_val = cross_analysis.max().max()
            max_locations = np.where(cross_analysis == max_val)
            if len(max_locations[0]) > 0:
                mercado_prob = cross_analysis.index[max_locations[0][0]]
                problema_prob = cross_analysis.columns[max_locations[1][0]]
                print(f"\n🎯 Combinación más problemática: {mercado_prob} + {problema_prob} ({max_val:,} llamadas repetidas)")

    # ==========================================
    # 6. VALIDACIÓN CONTRA TABLEAU
    # ==========================================

    print(f"\n✅ VALIDACIÓN FINAL:")
    print("-" * 20)
    print(f"¿Coincide con Tableau?")
    print(f"Total llamadas: {total_all_calls:,} (esperado: ~63,156)")
    print(f"FCR: {fcr_rate:.1f}% (esperado: ~78.7%)")
    print(f"Llamadas repetidas: {total_repeat_calls:,} (esperado: ~11,584)")
    print(f"Ratio recontacto: {repeat_rate:.1f}% (esperado: ~21.3%)")

    # ==========================================
    # 7. RETORNAR DATOS CORREGIDOS
    # ==========================================

    return {
        'df_corregido': df,
        'metricas_principales': {
            'total_calls': total_all_calls,
            'fcr_rate': fcr_rate,
            'repeat_rate': repeat_rate,
            'total_repeats': total_repeat_calls
        },
        'escalamiento': escalamiento_df,
        'por_problema': problem_analysis if 'problem_type' in df.columns else None,
        'por_mercado': market_analysis if 'market' in df.columns else None
    }

def generate_escalamiento_viz(escalamiento_df, output_path='escalamiento_plot.png'):
    """
    Crear visualización del patrón de escalamiento N1→N7
    """
    plt.figure(figsize=(12, 6))

    # Gráfico de línea para mostrar el patrón
    plt.plot(escalamiento_df['Contacto'], escalamiento_df['Llamadas'],
             marker='o', linewidth=2, markersize=8)

    plt.title('Patrón de Escalamiento: Llamadas Repetidas por Contacto', fontsize=14)
    plt.xlabel('Número de Contacto', fontsize=12)
    plt.ylabel('Número de Llamadas', fontsize=12)
    plt.grid(True, alpha=0.3)

    # Anotar valores
    for i, row in escalamiento_df.iterrows():
        plt.annotate(f"{row['Llamadas']:,}",
                    (row['Contacto'], row['Llamadas']),
                    textcoords="offset points",
                    xytext=(0,10),
                    ha='center')

    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"📊 Gráfico guardado en: {output_path}")

# FUNCIÓN PRINCIPAL PARA EJECUTAR TODO
def run_complete_analysis(df):
    """
    Ejecutar análisis completo corregido
    """
    print("🚀 INICIANDO ANÁLISIS COMPLETO CORREGIDO...")
    print("=" * 60)

    results = complete_corrected_analysis(df)

    # Generar visualización del escalamiento
    if results['escalamiento'] is not None and not results['escalamiento'].empty:
        generate_escalamiento_viz(results['escalamiento'])

    return results

# ==========================================
# EJECUCIÓN AUTOMÁTICA COMPLETA
# ==========================================

def clean_and_analyze_google_fiber():
    """
    Función principal que ejecuta TODO el proceso automáticamente
    """

    # 1. Cargar y limpiar datos
    print("📊 PASO 1: CARGANDO DATOS...")
    print("=" * 50)

    url_market1 = 'https://docs.google.com/spreadsheets/d/1T7irtn0ay9MfuhG_6y2jEbeT-zLuEQTbv1qNxkpBaTE/export?format=csv'
    url_market2 = 'https://docs.google.com/spreadsheets/d/1RE_MAKPt0JfWijbYCR32XFra9OGlmS2tLMxNs-X2NSU/export?format=csv'

    try:
        df1 = pd.read_csv(url_market1)
        df2 = pd.read_csv(url_market2)
        print(f"✅ Market 1: {len(df1)} registros")
        print(f"✅ Market 2: {len(df2)} registros")
    except Exception as e:
        print(f"❌ Error cargando datos: {e}")
        return None

    # 2. Estandarizar columnas
    print("\n🔧 PASO 2: ESTANDARIZANDO DATOS...")
    df1.columns = df1.columns.str.strip().str.lower().str.replace(' ', '_')
    df2.columns = df2.columns.str.strip().str.lower().str.replace(' ', '_')

    # 3. Unir datasets
    df = pd.concat([df1, df2], ignore_index=True)
    print(f"Dataset combinado: {len(df)} registros")

    # 4. Limpiar fechas
    print("\n📅 PASO 3: PROCESANDO FECHAS...")
    if 'date_created' in df.columns:
        df['date_created'] = pd.to_datetime(df['date_created'], errors='coerce')
        invalid_dates = df['date_created'].isnull().sum()
        if invalid_dates > 0:
            print(f"⚠️ Eliminando {invalid_dates} registros con fechas inválidas")
            df = df.dropna(subset=['date_created'])

    # 5. Renombrar columnas principales
    rename_mapping = {
        'new_type': 'problem_type',
        'new_market': 'market'
    }
    df = df.rename(columns=rename_mapping)

    # 6. EJECUTAR ANÁLISIS COMPLETO
    print("\n🚀 PASO 4: EJECUTANDO ANÁLISIS COMPLETO...")
    print("=" * 50)

    results = complete_corrected_analysis(df)

    # 7. Mostrar resumen final
    print("\n" + "="*60)
    print("🎯 RESUMEN EJECUTIVO FINAL")
    print("="*60)

    metrics = results['metricas_principales']
    print(f"📊 MÉTRICAS VALIDADAS CONTRA TABLEAU:")
    print(f"   ├─ Total Llamadas: {metrics['total_calls']:,}")
    print(f"   ├─ FCR Rate: {metrics['fcr_rate']:.1f}%")
    print(f"   ├─ Llamadas Repetidas: {metrics['total_repeats']:,}")
    print(f"   └─ Ratio Recontacto: {metrics['repeat_rate']:.1f}%")

    if results['por_problema'] is not None:
        print(f"\n🥇 PROBLEMA MÁS CRÍTICO:")
        worst_problem = results['por_problema'].iloc[0]
        print(f"   └─ {worst_problem.name}: {worst_problem['repeat_rate']:.1f}% tasa repetidas")

    if results['escalamiento'] is not None:
        print(f"\n📈 PATRÓN DE ESCALAMIENTO:")
        for _, row in results['escalamiento'].iterrows():
            print(f"   ├─ {row['Contacto']}: {row['Llamadas']:,} llamadas ({row['Porcentaje']:.1f}%)")

    print(f"\n✅ ANÁLISIS COMPLETADO EXITOSAMENTE!")

    return df, results

# ==========================================
# EJECUCIÓN INMEDIATA AL CARGAR SCRIPT
# ==========================================

if __name__ == "__main__":
    print("🎬 EJECUTANDO ANÁLISIS COMPLETO DE GOOGLE FIBER...")
    print("🔄 Procesando datos automáticamente...\n")

    # EJECUTAR TODO AUTOMÁTICAMENTE
    df_final, analysis_results = clean_and_analyze_google_fiber()

    if df_final is not None:
        print("\n💾 GUARDANDO RESULTADOS...")
        df_final.to_csv('google_fiber_analysis_complete.csv', index=False)
        print("📁 Archivo guardado: google_fiber_analysis_complete.csv")

        print("\n🎉 ¡PROCESO COMPLETADO!")
        print("🔍 Revisa los resultados impresos arriba para validar contra Tableau")
    else:
        print("❌ Error en el procesamiento")

# TAMBIÉN PUEDES EJECUTAR MANUALMENTE ASÍ:
# df, results = clean_and_analyze_google_fiber()
print("\n" + "="*60)
print("🚀 EJECUTANDO AUTOMÁTICAMENTE...")
print("="*60)

🎬 EJECUTANDO ANÁLISIS COMPLETO DE GOOGLE FIBER...
🔄 Procesando datos automáticamente...

📊 PASO 1: CARGANDO DATOS...
✅ Market 1: 450 registros
✅ Market 2: 450 registros

🔧 PASO 2: ESTANDARIZANDO DATOS...
Dataset combinado: 900 registros

📅 PASO 3: PROCESANDO FECHAS...

🚀 PASO 4: EJECUTANDO ANÁLISIS COMPLETO...
🔧 CORRIGIENDO TODAS LAS MÉTRICAS...
📞 Columna primera llamada: contacts_n
🔁 Columnas repetidas: ['contacts_n_1', 'contacts_n_2', 'contacts_n_3', 'contacts_n_4', 'contacts_n_5', 'contacts_n_6', 'contacts_n_7']

📊 MÉTRICAS PRINCIPALES (CORREGIDAS):
----------------------------------------
Total de llamadas: 63,156.0
├─ Primera llamada: 49,722.0
└─ Llamadas repetidas: 13,434.0
FCR Rate: 78.7%
Repeat Rate: 21.3%
Casos totales: 900
├─ Sin repetición: 252 (28.0%)
└─ Con repetición: 648 (72.0%)

🔍 ANÁLISIS DE ESCALAMIENTO POR CONTACTO:
---------------------------------------------
Contacto N1: 3,651.0 llamadas (27.2%)
Contacto N2: 2,295.0 llamadas (17.1%)
Contacto N3: 1,780.0 llamadas (

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df.to_csv('/content/drive/MyDrive/google_fiber_clean.csv', index=False)


Mounted at /content/drive


2. OPCION SIMPLE DE LIMPIEZA DE DATOS

In [ ]:
import pandas as pd

# 1. Cargar datos desde Google Sheets como CSV
url_market1 = 'https://docs.google.com/spreadsheets/d/1T7irtn0ay9MfuhG_6y2jEbeT-zLuEQTbv1qNxkpBaTE/export?format=csv'
url_market2 = 'https://docs.google.com/spreadsheets/d/1RE_MAKPt0JfWijbYCR32XFra9OGlmS2tLMxNs-X2NSU/export?format=csv'

df1 = pd.read_csv(url_market1)
df2 = pd.read_csv(url_market2)

# 2. Estandarizar nombres de columnas
df1.columns = df1.columns.str.strip().str.lower()
df2.columns = df2.columns.str.strip().str.lower()

# 3. Verificar estructura común
assert set(df1.columns) == set(df2.columns), "⚠️ Columnas no coinciden entre datasets"

# 4. Unir los dataframes
df = pd.concat([df1, df2], ignore_index=True)

# 5. Eliminar duplicados
df = df.drop_duplicates()

# 6. Verificar y convertir fecha
df['date_created'] = pd.to_datetime(df['date_created'], errors='coerce')

# 7. Eliminar filas con fechas nulas o contactos vacíos
df = df.dropna(subset=['date_created', 'contacts_n'])

# 8. Convertir columnas numéricas
contact_cols = ['contacts_n', 'contacts_n_1', 'contacts_n_2', 'contacts_n_3',
                'contacts_n_4', 'contacts_n_5', 'contacts_n_6', 'contacts_n_7']

df[contact_cols] = df[contact_cols].fillna(0).astype(int)

# 9. Renombrar columnas para claridad
df = df.rename(columns={
    'new_type': 'problem_type',
    'new_market': 'market'
})

# 10. Calcular columnas clave
repeat_cols = ['contacts_n_1', 'contacts_n_2', 'contacts_n_3',
               'contacts_n_4', 'contacts_n_5', 'contacts_n_6', 'contacts_n_7']

df['total_repeats'] = df[repeat_cols].sum(axis=1)
df['total_calls'] = df['contacts_n'] + df['total_repeats']
df['resolved_first_call'] = (df['total_repeats'] == 0).astype(int)  # 1 = resuelto en primera llamada

# 11. Crear columna de semana y mes para análisis temporal
df['week'] = df['date_created'].dt.to_period('W').astype(str)
df['month'] = df['date_created'].dt.to_period('M').astype(str)

# 12. Revisar tipos y valores únicos
print("✅ Datos limpios y listos. Vista previa:")
print(df.dtypes)
print(df.head())

# 13. Exportar CSV limpio
df.to_csv('google_fiber_llamadas_limpias.csv', index=False)


✅ Datos limpios y listos. Vista previa:
date_created           datetime64[ns]
contacts_n                      int64
contacts_n_1                    int64
contacts_n_2                    int64
contacts_n_3                    int64
contacts_n_4                    int64
contacts_n_5                    int64
contacts_n_6                    int64
contacts_n_7                    int64
problem_type                   object
market                         object
total_repeats                   int64
total_calls                     int64
resolved_first_call             int64
week                           object
month                          object
dtype: object
  date_created  contacts_n  contacts_n_1  contacts_n_2  contacts_n_3  \
0   2022-02-04         199            21             6            11   
1   2022-01-30          19             2             0             2   
2   2022-02-14          29             0             2             2   
3   2022-01-16         120             6          

Similitudes y diferencias en los 2 analisis

🔹 Similitudes

Ambos:

Cargan los dos datasets desde Google Sheets.

Estandarizan nombres de columnas.

Unifican los datasets en un solo DataFrame.

Limpian duplicados y nulos básicos.

Procesan fechas (date_created).

Normalizan columnas de contactos a enteros (contacts_n, contacts_n_1...).

Calculan métricas claves:

total_repeats

total_calls

resolved_first_call

Crean variables temporales (week, month en la simple; year, month, week, day_of_week en la avanzada).

Exportan el resultado como CSV limpio.

🔹 Diferencias clave
Aspecto	Versión simple	Versión mejorada
Carga de datos	Usa pd.read_csv directamente, sin validación.	Maneja errores con try/except y muestra el conteo de registros por mercado.
Columnas	assert para que ambas tablas tengan las mismas columnas. Si no coinciden, da error.	Más flexible: muestra diferencias y trabaja solo con columnas comunes.
Fechas	Convierte date_created con un único intento (pd.to_datetime).	Intenta múltiples formatos (%Y-%m-%d, %d/%m/%Y, etc.), reporta conversiones y elimina inválidas.
Duplicados	Solo elimina filas duplicadas completas.	Detecta duplicados por fecha, mercado y tipo de problema, y los agrega sumando contactos. Mucho más realista.
Contactos	Asume columnas fijas (contacts_n, contacts_n_1...contacts_n_7).	Busca dinámicamente todas las columnas que contengan "contacts_", escalable si el dataset cambia.
Categóricas	Solo renombra new_type → problem_type y new_market → market.	Limpia espacios, reemplaza nan, null, None y reporta los valores únicos para revisión.
Métricas	Calcula resolved_first_call (binario).	Además:

repeat_call_flag (llamadas repetidas)

FCR rate (%)

Repeat Call Rate (%) |
| Variables temporales | week y month. | Más completas: year, month, week, day_of_week, date_only, year_month, year_week. |
| Validación | Solo imprime dtypes y head(). | Hace análisis completo: rango de fechas, nulos restantes, estadísticas descriptivas. |
| Exportación | Exporta un CSV. | Exporta CSV + genera un reporte de calidad en TXT con distribución por mercado, tipo de problema, métricas de rendimiento y columnas disponibles. |
| Escalabilidad | Adecuada para datasets simples y controlados. | Robusta y adaptable a datasets heterogéneos y en producción. |

:: # *json promp para otras limpiezas*:

In [ ]:
{
  "data_cleaning_steps": [
    {
      "step": 1,
      "action": "load_data",
      "details": {
        "source": ["csv", "excel", "sql", "google_sheets"],
        "error_handling": "try/except",
        "log": ["rows", "columns"]
      }
    },
    {
      "step": 2,
      "action": "standardize_columns",
      "details": {
        "lowercase": true,
        "strip_spaces": true,
        "replace_spaces_with": "_"
      }
    },
    {
      "step": 3,
      "action": "check_structure",
      "details": {
        "validate_columns": true,
        "handle_differences": ["report", "keep_common"]
      }
    },
    {
      "step": 4,
      "action": "merge_datasets",
      "details": {
        "method": "concat",
        "ignore_index": true
      }
    },
    {
      "step": 5,
      "action": "analyze_nulls",
      "details": {
        "metrics": ["count", "percentage"],
        "strategy": ["drop", "impute", "keep"]
      }
    },
    {
      "step": 6,
      "action": "process_dates",
      "details": {
        "formats": ["%Y-%m-%d", "%d/%m/%Y", "%m/%d/%Y"],
        "convert_to": "datetime",
        "new_features": ["year", "month", "week", "day_of_week", "periods"]
      }
    },
    {
      "step": 7,
      "action": "process_numeric",
      "details": {
        "convert": "pd.to_numeric",
        "fillna": 0,
        "types": ["int", "float"],
        "outliers": "optional"
      }
    },
    {
      "step": 8,
      "action": "process_categorical",
      "details": {
        "clean_spaces": true,
        "lowercase": true,
        "replace_invalid": ["NaN", "null", "None"]
      }
    },
    {
      "step": 9,
      "action": "handle_duplicates",
      "details": {
        "subset": ["id", "date", "category"],
        "strategy": ["drop", "aggregate"]
      }
    },
    {
      "step": 10,
      "action": "generate_metrics",
      "details": {
        "totals": ["sum_columns"],
        "flags": ["binary_indicators"],
        "ratios": ["conversion_rate", "repeat_rate"]
      }
    },
    {
      "step": 11,
      "action": "final_validation",
      "details": {
        "check_nulls": true,
        "check_date_range": true,
        "summary": ["df.info", "df.describe"]
      }
    },
    {
      "step": 12,
      "action": "export_data",
      "details": {
        "format": ["csv", "excel", "db"],
        "filename": "dataset_clean_DATE.csv"
      }
    },
    {
      "step": 13,
      "action": "generate_report",
      "details": {
        "include": [
          "initial_vs_final_records",
          "null_handling_summary",
          "categorical_distribution",
          "key_metrics"
        ],
        "output": "txt"
      }
    }
  ]
}
